In [65]:
# importing the necessary modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.sas.sas7bdat import _column
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows',10000)
pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',None)

Cleaning the data. Removing the first two columns because they are blank and contains index and timestamp, which are not required for the analysis

In [66]:

df = pd.read_excel("SUDS.xlsx")

col = df.columns[0]

df.drop(columns=col, inplace = True)

Renaming the columns.

In [67]:
df = df.rename(columns = {'X1..What.do.you.like.best.about..OpenStreetsPGH....check.all.that.apply.':'Best_about_BikePGH'})
df = df.rename(columns = {'X2..Have.you.attended.OpenStreetsPGH.before...check.one.':'Attended_Before'})
df = df.rename(columns = {'X3..How.did.you.hear.about.OpenStreetsPGH...check.all.that.apply.':'How_you_heard'})
df = df.rename(columns = {'X4..What.are.your.primary.activities.at.OpenStreetsPGH..check.up.to.2.':'Primary_activities'})
df = df.rename(columns = {'X5..How.long.will.you.spend.at.OSP.today.':'Time_you_will_spend'})
df = df.rename(columns = {'X6..Do.you.typically.walk.for.exercise..not.including.OpenStreetsPGH..':'Walking_tendency'})
df = df.rename(columns = {'X7..Do.you.typically.bike.for.exercise..not.including.OpenStreetsPGH..':'Biking_tendency'})
df = df.rename(columns = {'X8..What.s.your.likelihood.of.walking.for.exercise.in.the.future.':'Likelihood_walking'})
df = df.rename(columns = {'X9..What.s.your.likelihood.of.biking.for.exercise.in.the.future.':'Likelihood_biking'})
df = df.rename(columns = {'X10..Do.you.typically.walk.for.transportation.trips..e.g..commute..errands..shop..visiting.friends..etc..':'Do_you_walk'})
df = df.rename(columns = {'X11..Do.you.typically.Bike.for.transportation.trips.':'Do_you_bike'})
df = df.rename(columns = {'X12..What.s.your.likelihood.of.walking.for.transportation...trips.in.the.future.':'Likelihood_walking'})
df = df.rename(columns = {'X13..What.s.your.likelihood.of.biking.for.transportation...trips.in.the.future.':'Likelihood_biking'})
df = df.rename(columns = {'X14..What.would.get.you.riding.a.bike.more.often.':'Reasons_for_biking'})
df = df.rename(columns = {'X15..Yes.or.no..did.you.know.that.the.nonprofit.organization..Bike.Pittsburgh..organizes.OpenstreetsPGH..':'How_you_heard'})
df = df.rename(columns = {'Did.you.bring.any.children.to.OSP..if.so.write.the.number.of.childres.next.to.the.corresponding.age.range....0.3.year.old.':'Children_0-3'})
df = df.rename(columns = {'Did.you.bring.any.children.to.OSP..if.so.write.the.number.of.childres.next.to.the.corresponding.age.range....4.8.year.old..':'Children_4-8'})
df = df.rename(columns = {'Did.you.bring.any.children.to.OSP..if.so.write.the.number.of.childres.next.to.the.corresponding.age.range....9.13.year.old.':'Children_9-13'})
df = df.rename(columns = {'Did.you.bring.any.children.to.OSP..if.so.write.the.number.of.childres.next.to.the.corresponding.age.range....14.17.year.old.':'Children_14-17'})
df = df.rename(columns = {'X17..Did.you.your.family.spend.any.money.or.do.you.plan.on.spending.any.money.today.at.OpenStreetsPGH.':'Money_spent_in_OSP'})
df = df.rename(columns = {'X18.Would.you.attend.another.OpenStreetsPGH.in.the.future.':'Attend_in_future'})
df = df.rename(columns = {'Race.or.Ethnicity':'Race'})
df = df.rename(columns = {'Gender.Identity':'Gender'})
df = df.rename(columns = {'Zip.Code':'ZipCode'})
df = df.rename(columns = {'Household.Income':'Income'})
df = df.rename(columns = {'city':'City'})
df = df.rename(columns = {'state':'State'})
df = df.rename(columns = {'latitude':'Latitude'})
df = df.rename(columns = {'longitude':'Longitude'})

In [68]:
# Printing the column names to get the clear picture

print(df.columns)

Index(['Best_about_BikePGH', 'Attended_Before', 'How_you_heard',
       'Primary_activities', 'Time_you_will_spend', 'Walking_tendency',
       'Biking_tendency', 'Likelihood_walking', 'Likelihood_biking',
       'Do_you_walk', 'Do_you_bike', 'Likelihood_walking', 'Likelihood_biking',
       'Reasons_for_biking', 'How_you_heard', 'Children_0-3', 'Children_4-8',
       'Children_9-13', 'Children_14-17', 'Money_spent_in_OSP',
       'Attend_in_future', 'Age', 'Race', 'Gender', 'ZipCode', 'Income',
       'City', 'State', 'Latitude', 'Longitude'],
      dtype='object')


Preparing the data for multiple regression. Mapping the variables to the correct values.
In case of variables which are in between some numerical values, I have taken up a calculated estimate.
Printing the unique names to get the clear picture of unique values in the columns.

In [69]:

df['Attended_Before'] = df['Attended_Before'].map({'This is my first time':0,
                                                   'once before':1,
                                                   '2-3 times':2,
                                                   '4-5 times':4})

print(df['Money_spent_in_OSP'].unique())

[nan 'Between $10 and $25' 'Between $25 and $50' 'NONE' 'Less than $10'
 'More than $50']


In [70]:

df['Money_spent_in_OSP'] = df['Money_spent_in_OSP'].map({'Between $10 and $25':20,
                                                         'Between $25 and $50':40,
                                                         'Less than $10':10,
                                                         'More than $50:':60})

print(df['Money_spent_in_OSP'].unique())

[nan 20. 40. 10.]


In [71]:
df['Race'] = df['Race'].map({'Caucasian ':'Caucasian',
                             'Irish Italan': 'Irish Italian',
                             'African-American ': 'African American',
                             'African American ':'African American',
                             'African-American': 'African American',
                             'Caucasian / non hispanic':'Caucasian',
                             'Asian ':'Asian',
                             'European mixed':'Hispanic',
                             'Aftican-Indian-American':'African American',
                             'Hispanic':'Hispanic',
                             'Latina':'Latina',
                             'Asian':'Asian',
                             'Black':'Black',
                             'African American/ caribbean': 'African American',
                             'White':'Hispanic',
                             })

print(df['Race'].unique())

[nan 'Caucasian' 'African American' 'Irish Italian' 'Hispanic' 'Asian'
 'Latina' 'Black']


In [72]:
df['Age'] = df['Age'].replace(199,19)

print(df['Age'].unique())

[12. 15. 16. 17. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.
 33. 34. 35. 36. 37. 38. 39. 40. 41. 42. 43. 45. 46. 47. 48. 49. 50. 52.
 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 69. 72. 75.
 86. 87. nan]


In [73]:

df['Gender'] = df['Gender'].replace('Non-binary/ other','Other')

print(df['Gender'].unique())

['Male' 'Female' nan 'Other']


In [74]:

df['Income'] = df['Income'].map({'25,000 - 49,999': 40000,
                                 '<25,000':20000,
                                 '75,000 - 99,999': 85000,
                                 '50,000 - 74,999': 65000,
                                 '125,000 - 149,000': 135000,
                                 '100,000 - 124,999': 115000,
                                 '>150,000': 170000})

print(df['Income'].unique())

print(df['State'].unique())

[    nan  40000.  20000.  85000.  65000. 135000. 115000. 170000.]
[nan 'PA' 'OH' 'NY']


Combining the relevant columns for multiple regression, and dropping all the NaN values so that we get a clear dataframe of objects where each and every data is present

In [75]:
regression_dataframe = pd.DataFrame([df['Attended_Before'], df['Money_spent_in_OSP'],
                                     df['Race'], df['Age'], df['Gender'],df['Income']])

regression_dataframe = regression_dataframe.transpose()

regression_dataframe = regression_dataframe.dropna()

print(regression_dataframe)

    Attended_Before Money_spent_in_OSP              Race Age  Gender  Income
2                 1                 20         Caucasian  15  Female   40000
7                 2                 10     Irish Italian  17    Male   85000
14                0                 20             Asian  22  Female   20000
15                0                 20             Asian  22    Male   20000
24                0                 20         Caucasian  23  Female   20000
28                0                 40          Hispanic  25    Male  135000
31                4                 10         Caucasian  26    Male   65000
33                0                 40  African American  26    Male  135000
35                2                 10         Caucasian  27  Female  115000
36                0                 10          Hispanic  27  Female   65000
37                0                 20            Latina  27  Female   40000
38                0                 20  African American  27  Female   40000

Preparing the dummy variables(Race and Gender).
Creating the first basetable(DataFrame to be used for the regression) by dropping the unnecessary columns.

In [76]:
dummy_race = pd.get_dummies(regression_dataframe['Race'])
dummy_gender = pd.get_dummies(regression_dataframe['Gender'])

basetable = pd.concat([regression_dataframe, dummy_race, dummy_gender], axis = 1, sort = False)
basetable = basetable.drop(columns = ['Race', 'Gender','Attended_Before'])
print(basetable)

    Money_spent_in_OSP Age  Income  African American  Asian  Caucasian  \
2                   20  15   40000                 0      0          1   
7                   10  17   85000                 0      0          0   
14                  20  22   20000                 0      1          0   
15                  20  22   20000                 0      1          0   
24                  20  23   20000                 0      0          1   
28                  40  25  135000                 0      0          0   
31                  10  26   65000                 0      0          1   
33                  40  26  135000                 1      0          0   
35                  10  27  115000                 0      0          1   
36                  10  27   65000                 0      0          0   
37                  20  27   40000                 0      0          0   
38                  20  27   40000                 1      0          0   
43                  20  28   85000    

In [77]:
# Considered all the columns except the dependent variable
X = basetable.drop(columns = ['Income'])
# Our dependent variables
y = basetable['Income']
# creating the train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=21)

In [78]:

# Create the regressor: reg_all
reg_all = LinearRegression()

# Fit the regressor to the training data
reg_all.fit(X_train,y_train)

# Predict on the test data: y_pred
y_pred = reg_all.predict(X_test)

# Compute and print R^2 and RMSE
print("Rsquare: {}".format(reg_all.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error: {}".format(rmse))

Rsquare: 0.20835881443278048
Root Mean Squared Error: 45760.254435371724
